In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2719/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_10_9_5,0.998830,0.998847,0.997150,0.998525,0.344447,0.633915,0.512670,0.576858,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
1,model_8_9_10,0.998822,0.996250,0.996833,0.996535,0.346791,1.113484,0.811284,0.971270,"Hidden Size=[25, 15], regularizer=0.1, learnin..."
2,model_8_9_9,0.998819,0.996361,0.996993,0.996666,0.347847,1.080681,0.770261,0.934602,"Hidden Size=[25, 15], regularizer=0.1, learnin..."
3,model_8_9_11,0.998818,0.996138,0.996673,0.996403,0.348136,1.146788,0.852322,1.008214,"Hidden Size=[25, 15], regularizer=0.1, learnin..."
4,model_10_9_6,0.998816,0.998604,0.996565,0.998217,0.348736,0.767596,0.617864,0.697136,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
254,model_12_8_1,0.994410,0.992333,0.997138,0.994811,1.645823,3.123269,0.916727,2.084897,"Hidden Size=[35], regularizer=0.1, learning_ra..."
267,model_13_9_3,0.994126,0.997182,0.996854,0.997138,1.729456,1.702317,0.591665,1.179661,"Hidden Size=[10, 5], regularizer=0.01, learnin..."
268,model_16_9_1,0.994089,0.998119,0.998050,0.998231,1.740520,0.883097,0.886555,0.884722,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
278,model_16_8_3,0.993990,0.996339,0.991195,0.993725,1.769548,0.741802,2.208151,1.431848,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
